![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [4]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [5]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [ ]:
# Celda 1


### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 2


### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 3


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [6]:
# Celda 6
# Importación y definición de modelo XGBRegressor
from xgboost import XGBRegressor
clf = XGBRegressor()
clf

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [7]:
# Entrenamiento (fit) y desempeño del modelo XGBRegressor
from sklearn import metrics
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [8]:
#Cálculo de MSE
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 2629001.844928506


In [9]:
rmse = sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 1621.4197004256812


In [10]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MAE: 1186.634392366123


In [12]:
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

R^2: 0.8281265175296673


#### Análisis:
El modelo XGBoost de regresión con los parámetros por defecto nos arroja un MSE (Error cuadrático medio) de 2629001.84, un RMSE (raíz cuadrada del MSE) de 1621.41, un MAE de 1186.63 y un R2 de 0.828 . Al analizar estas medidas de desempeño con el modelo calibrado del punto siguiente, nos damos cuenta que este modelo tiene un desempeño levemente inferior evaluado a través de los mismos indicadores.

### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [27]:
# Celda 7
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Definimos los valores a probar para cada parámetro
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'gamma': [0, 1, 5],
    'colsample_bytree': [0.3, 0.7, 1]
}

In [28]:
# Creamos un objeto XGBRegressor
modelo1 = XGBRegressor()

# Creamos un objeto GridSearchCV
grid_search = GridSearchCV(modelo1, param_grid, cv=5)

In [29]:
# Realizamos la búsqueda en cuadrícula para encontrar los mejores parámetros utilizando validación cruzada
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
 

In [30]:
# Se muestran los mejores parámetros encontrados
print(grid_search.best_params_)

{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1}


In [31]:
clf = XGBRegressor(learning_rate=0.1,gamma=0,colsample_bytree=0.7)
clf

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [32]:
# Entrenamiento (fit) y desempeño del modelo XGBRegressor
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [33]:
#Cálculo de MSE
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 2395512.948914845


In [34]:
rmse = sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 1547.7444714534906


In [35]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MAE: 1133.055212712453


In [36]:
r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')

R^2: 0.8433910749712816


#### Análisis
El modelo XGBoost de regresión calibrando los parámetros learning rate, gamma y colsample_bytree, a través del método GridSearchCV que realiza una búsqueda en cuadrícula para encontrar los mejores parámetros utilizando validación cruzada, nos arroja un MSE (Error cuadrático medio) de 2395512.94, un RMSE (raíz cuadrada del MSE) de 1547.74, un MAE de 1186.63 y un R2 de 0.843 . 

Al analizar estas medidas de desempeño con el modelo sin calibrar del punto anterior, nos damos cuenta que este modelo tiene un desempeño superior al evaluado, dado que los parámetros learning rate, gamma y colsample_bytree son tres parámetros importantes en un modelo XGBoost de regresión.

Por una parte, learning_rate controla el tamaño de los pasos que el modelo da al actualizar los pesos durante el entrenamiento. Un valor más bajo hace que el modelo aprenda más lentamente, lo que puede mejorar la precisión pero aumentar el tiempo de entrenamiento. 

Así mismo, gamma controla la complejidad del modelo al regular la poda de los árboles. Un valor más alto hace que el modelo sea más conservador al podar los árboles, lo que puede reducir el sobreajuste pero aumentar el sesgo.

Finalmente, colsample_bytree controla la fracción de características que se utilizan en cada árbol. Un valor más bajo hace que cada árbol utilice menos características, lo que puede mejorar la generalización pero disminuir la precisión del modelo

Encontrar los mejores valores de estos parámetros en la calibración del modelo, garantiza un mejor desempeño -como hemos evidenciado con los resultados de este punto-.

### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
